# Paper: Energy Economics of the Green Transition

Defining two vectors with the different values of hydro and wind

In [57]:
pip install read


Note: you may need to restart the kernel to use updated packages.


In [61]:
# importing packages
import numpy as np
import pandas as pd
import os
from underliggende_koder import *
import read



#defining path to data
# Din fulde sti
full_path = r'C:\Users\CAAL\Dropbox\Polit\Kandidat\4. semester\EE of the GT\Greenland\Codes\Greenland\E44_Data.xlsx'

# Opret separate komponenter af stien
path_components = full_path.split(os.sep)

# Brug os.path.join til at oprette stien
path_to_data = os.path.join(*path_components)

print("Sti til data:", path_to_data)


Sti til data: C:Users\CAAL\Dropbox\Polit\Kandidat\4. semester\EE of the GT\Greenland\Codes\Greenland\E44_Data.xlsx


In [62]:
kwargs = {'variables': ['Fundamentals', 'Load', 'Generators_Other'],
        'maps': ['Generators_Categories','hMaps'],
        'variable2D': ['HourlyVariation','Generators_FuelMix'],
        'scalars': ['Scalars']}

db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

AttributeError: module 'read' has no attribute 'dbFromWB'

In [39]:
# Defining vectors for wind and hydro
vec_wind = np.linspace(0, 55, 5)
vec_hydro = np.linspace(45, 100, 5)

# printing vectors
print("Vector for wind:", vec_wind)
print("Vector for hydro:", vec_hydro)


Vector for wind: [ 0.   13.75 27.5  41.25 55.  ]
Vector for hydro: [ 45.    58.75  72.5   86.25 100.  ]


In [40]:
# Setting
E2P = 4 # The "energy storage to power"-ratio/storage duration

maxcap = 100

# Defining two loops
loop = pd.Index(range(5), name = 'loop')
loop2 = pd.Index(range(5), name = 'loop2')

# Defining parameters
v0_ECap = pd.Series(0.001, index = getTechs(['Storage'], m.db))
vT_ECap = pd.Series(maxcap, index = v0_ECap.index)
v0_SCap = v0_ECap.mul(E2P)
vT_SCap = vT_ECap.mul(E2P)

grid_ECap = adjMultiIndex.addGrid(v0_ECap,vT_ECap,loop,'GeneratingCapacity').astype(int)
grid_SCap = adjMultiIndex.addGrid(v0_SCap,vT_SCap,loop2,'sCap').astype(int)

NameError: name 'm' is not defined

In [ ]:
def grid(v0,vT,index,gridtype='linear',phi=1):
""" If v0, vT are 1d numpy arrays, returns 2d array. If scalars, returns 1d arrays. """
    if gridtype == 'linear': 
        return np.linspace(v0,vT,len(index))   
    elif gridtype=='polynomial':
        return np.array([v0+(vT-v0)*((i-1)/(len(index)-1))**phi for i in range(1,len(index)+1)])

IndentationError: expected an indented block (2742015541.py, line 2)

In [ ]:
def addGrid(v0, vT,index,name,gridtype = 'linear', phi = 1,sort_levels=None, sort_index = False):
""" NB: Make sure that v0 and vT are sorted similarly (if they are defined over indices) """
if sort_index:
v0 = v0.sort_index()
vT = vT.sort_index()
if isinstance(v0,pd.Series):
    return pd.DataFrame(grid(v0,vT,index,gridtype=gridtype,phi=phi).T, index = v0.index, columns = index).stack().rename(name).reorder_levels(index.names+v0.index.names if sort_levels is None else sort_levels)
else:
    return pd.Series(grid(v0,vT,index,gridtype=gridtype,phi=phi), index = index,name=name)

IndentationError: expected an indented block (3621522988.py, line 2)

In [ ]:
df_E = lpCompiler.sortAll(addGrid(grid_ECap, grid_ECap, loop2, 'GeneratingCap_E').unstack('id'))
df_S = lpCompiler.sortAll(addGrid(grid_SCap, grid_SCap, loop, 'sCap').unstack('id'))
df_E.index = df_E.index.to_flat_index().rename('loop')
df_S.index = df_S.index.to_flat_index().rename('loop')
grid_ECap_1d = df_E.stack().rename('GeneratingCap_E')
grid_SCap_1d = df_S.stack().rename('GeneratingCap_H')
loop = df_E.index


In [ ]:

extract = ['marginalSystemCosts','Generation','charge','discharge','capturePrice','marketValueFactor','capacityFactor','marginalEconomicValue','hourlyEmissions','Emissions','consumerSurplus','producerSurplus','Welfare']


In [ ]:

solution = m.loopSolveExtract(loop, [grid_ECap_1d, grid_SCap_1d], extract)